In [19]:
library(boot)

In [20]:
n.samples = 50
n.sets = 10
n.resamples = 100
# n.repetitions = 10

## Determining Distribution Equality Using a CI of the Bootstrapped KS Statistic

We sample from two distributions, $A$ and $B$, and attempt to bootstrap a CI on the KS statistic.

Let $X_{A,i}$ be a vector of N samples from distribution A, for $i \in \{1..N_{sets}\}$. "A" is a stand-in for Python simulation code, "B" for the IDL. The "sets" are a stand-in for simulation runs.

Let $K_{j,k}$ be the KS statistic applied to sample vectors j and k.

### Test 1

This is one method that was suggested in our call. I am not entirely sure it is implemented as described. The expectation is that, since sets of samples are from the same distribution, the test would confirm that the distributions are equivalent.

A and B ~ N(0,1)

We bootstrap two 95% confidence intervals: 

$CI_1$ is constructed by sampling from $K_{j,k}, \forall j \in X_{A,i}$, $\forall k \in X_{A,i}$

$CI_2$ is constructed by sampling from $K_{j,k}, \forall j \in X_{B,i}$, $\forall k \in X_{B,i}$

We run this process repeatedly, and find that the CIs overlap in 40-60% of trials. This does not appear to be successful.

In [16]:
calculate.mean.for.bootstrap <- function(data, indices) {
    return(mean(data[indices]))
}

cis.overlap <- function(ci1, ci2) {
    return(ci1[1] > ci2[1] && ci1[1] < ci2[2] || ci2[1] > ci1[1] && ci2[1] < ci1[2])
}

In [17]:
repetition.results <- vector("numeric", n.repetitions)
for(x in 1:n.repetitions) {
    # Generate X.A and X.B
    X.A <- rnorm(n.samples * n.sets)
    X.A <- matrix(X.A, nrow=n.sets, byrow=TRUE)
    X.B <- rnorm(n.samples * n.sets)
    X.B <- matrix(X.B, nrow=n.sets, byrow=TRUE)
    
    # Calculate KS statistics for all pairs of sets
    # Generate X.A and X.B
    X.A <- rnorm(n.samples * n.sets)
    X.A <- matrix(X.A, nrow=n.sets, byrow=TRUE)
    X.B <- rnorm(n.samples * n.sets)
    X.B <- matrix(X.B, nrow=n.sets, byrow=TRUE)
    
    # Calculate KS statistics for all pairs of sets
    K.A.A <- vector("numeric", n.sets * (n.sets - 1))
    K.B.B <- vector("numeric", n.sets * (n.sets - 1))
    counter <- 1
    for(i in 1:n.sets) {
        for(j in 1:n.sets) {
            if(i != j) {
                K.A.A[counter] <- as.numeric(ks.test(X.A[i,], X.A[j,])$statistic)
                K.B.B[counter] <- as.numeric(ks.test(X.B[i,], X.B[j,])$statistic)
                counter = counter + 1
            }
        }
    }
    
    # Boostrap two CIs
    boot.res <- boot(data=K.A.A, statistic=calculate.mean.for.bootstrap, R=n.resamples)
    ci.res <- boot.ci(boot.res, type="bca")
    K.A.A.ci <- c(ci.res$bca[4], ci.res$bca[5])
    
    boot.res <- boot(data=K.B.B, statistic=calculate.mean.for.bootstrap, R=n.resamples)
    ci.res <- boot.ci(boot.res, type="bca")
    K.B.B.ci <- c(ci.res$bca[4], ci.res$bca[5])
    
    # Check if the CIs overlap
    repetition.results[x] <- cis.overlap(K.A.A.ci, K.B.B.ci)
}

mean(repetition.results)

[1] 0.41

In [18]:
# If we sample from two normal distributions with different means and variances, we get roughly the same results

repetition.results <- vector("numeric", n.repetitions)
for(x in 1:n.repetitions) {
    # Generate X.A and X.B
    X.A <- rnorm(n.samples * n.sets)
    X.A <- matrix(X.A, nrow=n.sets, byrow=TRUE)
    X.B <- rnorm(n.samples * n.sets, mean=5, sd=3)
    X.B <- matrix(X.B, nrow=n.sets, byrow=TRUE)
    
    # Calculate KS statistics for all pairs of sets
    # Generate X.A and X.B
    X.A <- rnorm(n.samples * n.sets)
    X.A <- matrix(X.A, nrow=n.sets, byrow=TRUE)
    X.B <- rnorm(n.samples * n.sets)
    X.B <- matrix(X.B, nrow=n.sets, byrow=TRUE)
    
    # Calculate KS statistics for all pairs of sets
    K.A.A <- vector("numeric", n.sets * (n.sets - 1))
    K.B.B <- vector("numeric", n.sets * (n.sets - 1))
    counter <- 1
    for(i in 1:n.sets) {
        for(j in 1:n.sets) {
            if(i != j) {
                K.A.A[counter] <- as.numeric(ks.test(X.A[i,], X.A[j,])$statistic)
                K.B.B[counter] <- as.numeric(ks.test(X.B[i,], X.B[j,])$statistic)
                counter = counter + 1
            }
        }
    }
    
    # Boostrap two CIs
    boot.res <- boot(data=K.A.A, statistic=calculate.mean.for.bootstrap, R=n.resamples)
    ci.res <- boot.ci(boot.res, type="bca")
    K.A.A.ci <- c(ci.res$bca[4], ci.res$bca[5])
    
    boot.res <- boot(data=K.B.B, statistic=calculate.mean.for.bootstrap, R=n.resamples)
    ci.res <- boot.ci(boot.res, type="bca")
    K.B.B.ci <- c(ci.res$bca[4], ci.res$bca[5])
    
    # Check if the CIs overlap
    repetition.results[x] <- cis.overlap(K.A.A.ci, K.B.B.ci)
}

mean(repetition.results)

[1] 0.393